In [9]:
import os
os.environ["OPENAI_API_KEY"] = "NA" # This satisfies the validator
os.environ["OTEL_SDK_DISABLED"] = "true"


# install the necessary libraries 
# !pip install crewai crewai_tools langchain_community
#pip install nest_asyncio
# Warning control
import warnings
warnings.filterwarnings('ignore')

import nest_asyncio
nest_asyncio.apply()


In [10]:
llm = "ollama/llama3.2"
llm = "ollama/mistral"


In [11]:

from crewai import Agent, Task, Crew, LLM
from crewai.flow.flow import Flow, listen, start
from crewai_tools import DirectoryReadTool, FileReadTool
import re
import os
import sys

class DynamicKnowledgeFlow(Flow):
    @start()
    def list_files(self):
        print("--- Step 1: Scanning Directory ---")
        dir_tool = DirectoryReadTool(directory='./knowledge')
        files_string = dir_tool.run()
        
        # IMPROVED PARSING LOGIC:
        # DirectoryReadTool returns formatted string like:
        # "File paths: \n-./knowledge/file1.txt\n-./knowledge/file2.txt"
        # Note: File paths are prefixed with "-" (dash)
        lines = [f.strip() for f in files_string.split('\n') if f.strip()]
        
        # Filter out header lines and extract actual file paths
        file_paths = []
        for line in lines:
            # Skip header lines like "File paths:", "Files:", etc.
            if line.lower().startswith('file') and ':' in line:
                continue
            
            # Remove dash prefix if present (DirectoryReadTool uses "-./path" format)
            path = line.strip()
            if path.startswith('-'):
                path = path[1:].strip()
            
            # Remove any leading/trailing quotes
            path = path.strip('"\'')
            
            # Check if line looks like a file path
            if path.startswith('./') or path.startswith('/') or '/' in path:
                # Validate path exists or is a valid relative path
                if os.path.exists(path) or path.startswith('./'):
                    file_paths.append(path)
        
        print(f"Found {len(file_paths)} files: {file_paths}")
        return file_paths[0] if file_paths else None

    @listen(list_files)
    def read_files_dynamically(self, first_file_path):
        """Step 2: Read file content. Step 3 will summarize it with a CrewAI agent."""
        if not first_file_path:
            return "No files found in directory."
        
        if not os.path.exists(first_file_path):
            return f"File not found: {first_file_path}"
            
        print(f"--- Step 2: Reading file: {first_file_path} ---")
        
        try:
            file_tool = FileReadTool(file_path=first_file_path)
            content = file_tool.run()
            print(content[:500] + "..." if len(content) > 500 else content)
            # Plain string (path + content) for next step
            return f"FILE_PATH:{first_file_path}\n\nCONTENT:\n{content}"
        except Exception as e:
            return f"FILE_PATH:{first_file_path}\n\nERROR:{str(e)}"

    @listen(read_files_dynamically)
    def summarize_content(self, read_result):
        """Step 3: Run a CrewAI agent to summarize the file content."""
        if "ERROR:" in read_result or read_result == "No files found in directory.":
            return read_result
        
        # Parse CONTENT from previous step
        if "CONTENT:\n" in read_result:
            content = read_result.split("CONTENT:\n", 1)[1]
        else:
            content = read_result
        
        # Truncate very long content to avoid token limits (optional)
        max_chars = 12000
        if len(content) > max_chars:
            content = content[:max_chars] + "\n\n[... truncated for summary ...]"
        
        print("--- Step 3: Summarizing with CrewAI agent ---")
        
        summarizer_llm = LLM(model=llm)
        summarizer_agent = Agent(
            role="Summarizer",
            goal="Provide clear, concise summaries that capture the main points and key information.",
            backstory="You are an expert at reading long texts and distilling them into accurate, useful summaries.",
            llm=summarizer_llm,
            verbose=True,
        )
        summary_task = Task(
            description="Summarize the following text. Be concise; capture the main points, key facts, and any important conclusions or recommendations.\n\nText to summarize:\n{{content}}",
            agent=summarizer_agent,
            expected_output="A concise summary paragraph (or bullet points) with the main ideas and key information.",
        )
        crew = Crew(agents=[summarizer_agent], tasks=[summary_task])
        crew_result = crew.kickoff(inputs={"content": content})
        # Coerce to plain str so flow return value never triggers Rich recursion
        summary = crew_result.raw if hasattr(crew_result, "raw") else str(crew_result)
        summary = str(summary) if summary is not None else ""
        print("\n--- Summary ---")
        sys.stdout.write(summary + "\n")
        sys.stdout.flush()
        return summary

In [12]:
# Execute the Flow: list_files -> read_files_dynamically -> summarize_content (CrewAI agent)
import sys
flow = DynamicKnowledgeFlow()
result = flow.kickoff()
# # Extract plain string only (no str(result) on wrapper — avoids Rich/display RecursionError)
# summary_str = getattr(result, "output", None) or getattr(result, "result", None)
# if summary_str is None and isinstance(result, str):
#     summary_str = result
# if summary_str is None:
#     summary_str = "(summary not available)"
# sys.stdout.write("\n=== Final result (summary) ===\n")
# sys.stdout.write(str(summary_str) + "\n")
# sys.stdout.flush()



Flow started with ID: a62581c9-f40c-47c7-8706-8d0da7933556

Flow started with ID: a62581c9-f40c-47c7-8706-8d0da7933556

--- Step 1: Scanning Directory ---
Using Tool: List files in directory


╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: DynamicKnowledgeFlow                                                                                     │
│  ID: a62581c9-f40c-47c7-8706-8d0da7933556                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The Batch | DeepLearning.AI | AI News & Insights ✨ New course! Enroll in Agent Skills with Anthropic Explore 
Courses AI Newsletter The Batch Andrew's Letter Data Points ML Research Blog ✨ AI Dev x SF 26 Community Forum 
Events Ambassadors Ambassador Spotlight Resources Membership Start Learning What Matters in AI Right Now Subscribe 
First Name Last Name Where do you live? Select a country What is your role? Please select Keep me updated on the 
latest news, events, and courses Subscribe 🗞️   Stay ...

--- Step 3: Summarizing with CrewAI agent ---

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer                                                                                              │
│                                                                                                                 │
│  Task: Summarize the following text. Be concise; capture the main points, key facts, and any important          │
│  conclusions or recommendations.                                                                                │
│                                                                                                                 │
│  Text to summarize:                                                                                             │
│  {The Batch | DeepLearning.AI | AI News & Insights ✨ New course! Enroll in Agent Skills with Anthropic         │
│  Explore Courses AI Newsletter The Batch Andrew's Letter Data Points ML Research Blog ✨ AI Dev x SF 26         │
│  Community Forum Events Ambassadors Ambassador Spotlight Resources Membership Start Learning What Matters in    │
│  AI Right Now Subscribe First Name Last Name Where do you live? Select a country What is your role? Please      │
│  select Keep me updated on the latest news, events, and courses Subscribe 🗞️   Stay updated with weekly AI News  │
│  and Insights delivered to your inbox Weekly Issues Andrew's Letters Data Points ML Research Business Science   │
│  Culture Hardware AI Careers About Jan 30, 2026 Agents Go Shopping, Intelligence Redefined, Better Text in      │
│  Pictures, Higher Engagement Means Worse Alignment The Batch AI News and Insights: U.S. policies are driving    │
│  allies away from using American AI technology. Read more Popular Articles Introducing DeepLearning.AI Pro Oct  │
│  31, 2025 1 min read Reinforcement Learning Heats Up, White House Orders Muscular AI Policy, Computer Use       │
│  Gains Momentum, Fine Control of Fine-Tuning Jan 29, 2025 13 min read DeepSeek Sharpens Its Reasoning:          │
│  DeepSeek-R1, an affordable rival to OpenAI’s o1 Jan 22, 2025 4 min read Jan 23, 2026 Self-Driving Reasoning    │
│  Models, ChatGPT Adds Ads, Apple’s Deal with Google, 3D Generation Pronto The Batch AI News and Insights: How   │
│  can businesses go beyond using AI for incremental efficiency gains to create transformative impact? Jan 16,    │
│  2026 Governments vs. Grok, Meta Buys Agent Tech, Healthcare Chatbots, Limits of AI-Powered Retrieval The       │
│  Batch AI News and Insights: Many people are fighting the growth of data centers because they could increase    │
│  CO2 emissions, electricity prices, and water use. Jan 09, 2026 LLMs Go To Confession, Automated Scientific     │
│  Research, What Copilot Users Want, Reasoning For Less The Batch AI News and Insights: We just launched a       │
│  course that shows people who have never coded before, in less than 30 minutes, how to describe an idea for an  │
│  app and build it using AI. Jan 02, 2026 New Year Special! Hopes for 2026 from David Cox, Adji Bousso Dieng,    │
│  Juan M. Lavista Ferres, Tanmay Gupta, Pengtao Xie, Sharon Zhou The Batch AI News and Insights: Happy 2026!     │
│  Will this be the year we finally achieve AGI? I’d like to propose a new version of the Turing Test, which      │
│  I’ll call the Turing-AGI Test, to see if we’ve achieved this. Dec 26, 2025 Top Stories of 2025! Big AI         │
│  Poaches Talent, Reasoning Models Boost Performance, Agents Write Code, Data Centers Drive GDP, China Turns     │
│  the Tables The Batch AI News and Insights: Another year of rapid AI advances has created more opportunities    │
│  than ever for anyone — including those just entering the field — to build software. Dec 17, 2025 OpenAI’s      │
│  Answer to Gemini 3, Runway’s Interactive Worlds, Disney

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The provided text consists of various articles and updates from "The Batch" newsletter focusing on AI-related  │
│  topics such as new courses, events, policy changes, technological advancements, and concerns about data        │
│  centers' impact on the environment. Some notable highlights include the introduction of DeepLearning.AI Pro,   │
│  the launch of a course for beginners to build an app using AI, and the development of self-driving reasoning   │
│  models, among others. The text also discusses trust issues in AI among Americans and parts of Europe, as well  │
│  as concerns about AI speculative bubbles and potential risks such as lethal drones and AI psychosis.           │
│  Additionally, there are mentions of partnerships between companies like Google, Microsoft, Universal Music     │
│  Group, and OpenAI, and developments in areas like robotics, reasoning models, and fine-tuning.                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Summary ---
The provided text consists of various articles and updates from "The Batch" newsletter focusing on AI-related topics such as new courses, events, policy changes, technological advancements, and concerns about data centers' impact on the environment. Some notable highlights include the introduction of DeepLearning.AI Pro, the launch of a course for beginners to build an app using AI, and the development of self-driving reasoning models, among others. The text also discusses trust issues in AI among Americans and parts of Europe, as well as concerns about AI speculative bubbles and potential risks such as lethal drones and AI psychosis. Additionally, there are mentions of partnerships between companies like Google, Microsoft, Universal Music Group, and OpenAI, and developments in areas like robotics, reasoning models, and fine-tuning.


Output()